# MRIO Workshop

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1.4 & 2 Load and clean data

### WIOT Data

In [2]:
lookup = pd.read_excel('Data/WIOT/wiot95_row_apr12.xlsx', index_col=None, header=None).iloc[6:,:4].set_index(1)
lookup[2].fillna('TOT', inplace=True)
lookup.columns = ['sector_code', 'country', 'sector']

In [3]:
lookup.head()

,sector_code,country,sector
1,,,
"Agriculture, Hunting, Forestry and Fishing",AtB,AUS,c1
Mining and Quarrying,C,AUS,c2
"Food, Beverages and Tobacco",15t16,AUS,c3
Textiles and Textile Products,17t18,AUS,c4
"Leather, Leather and Footwear",19,AUS,c5


In [4]:
# function for WIOT Data

def import_wiot(year):
    file_loc = 'Data/WIOT/wiot' + str(year)[2:] + '_row_apr12.xlsx'
    
    # import data
    df = pd.read_excel(file_loc, index_col=None, header=None).iloc[4:,2:]
    df[1644].fillna('Total', inplace=True)
    df[2].fillna('TOT', inplace=True)

    #set columns
    c1 = df.iloc[0].tolist()
    c1[0] = 'country'
    c2 = df.iloc[1].tolist()
    c2[1] = 'sector'
    df.columns = [c1, c2]

    # set index
    df = df.drop([4, 5], axis=0).set_index([('country', 'TOT'), (np.nan, 'sector')]).fillna(0)
    
    return(df)

In [5]:
year_list = []
for i in range(1995, 2010):
    year = i
    year_list.append(year)

In [6]:
wiot_list = []
for i in year_list:
    df = import_wiot(i)
    wiot_list.append(df)

In [7]:
# Z, V and Y
Z = []
V = []
Y = []

col_list = []
for i in range(1, 36):
    c = 'c' + str(i)
    col_list.append(c)
    
for i in range(len(wiot_list)):
    z=wiot_list[i].drop('TOT', axis=0).swaplevel(axis=1).loc[:, col_list].swaplevel(axis=1)
    Z.append(z)
    
    v=wiot_list[i].swaplevel(axis=0).loc[['r99', 'r60', 'r61', 'r62', 'r64', 'r69']].swaplevel(axis=1).loc[:, col_list].swaplevel(axis=1)
    V.append(v)
    
    y=wiot_list[i].drop('TOT', axis=0).swaplevel(axis=1)[['c37', 'c38', 'c39', 'c41', 'c42']]
    Y.append(y)

c:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 3. Load the extension data

In [8]:
# country list 

country_list = ['AUS','AUT','BEL','BGR','BRA','CAN','CHN','CYP','CZE','DEU','DNK','ESP','EST','FIN','FRA','GBR','GRC','HUN','IDN','IND','IRL','ITA','JPN','KOR','LTU','LUX','LVA','MEX','MLT',
                'NLD','POL','PRT','ROU','RUS','SVK','SVN','SWE','TUR','TWN','USA','ROW']

country = np.array(country_list)

l1 = lookup[['sector']].drop_duplicates()
l1 = l1.loc[l1.sector.str[0] == 'c']

### Energy

F_NRG

In [9]:
# function for Energy Data --> make array of vecotrs containing total energy use per year per sector

def import_energy(year, country_list):
    array_list = []
    df_list = []
    sheet = str(year)
    
    country = country_list[0]
    file_loc = 'Data/EAS-EU/' + str(country) + '_EU_May12.xls'
    # import data
    df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
    df.index = df.index.droplevel(level=1)
    df = df.join(l1, how='right')
    df['energy'] = df['TOTAL']
    df['country'] = str(country)
    df = df[['country', 'sector', 'energy']].fillna(0)
       
    for i in range(1, len(country_list)):
        country = country_list[i]
        file_loc = 'Data/EAS-EU/' + str(country) + '_EU_May12.xls'
        # import data
        temp = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
        temp.index = temp.index.droplevel(level=1)
        temp = temp.join(l1, how='right')
        temp['energy'] = temp['TOTAL']
        temp['country'] = str(country)
        temp = temp[['country', 'sector', 'energy']].fillna(0)
        
        df = df.append(temp)
    
    df=df.set_index(['country', 'sector']).T      
    return(df)

In [10]:
#array_list = []
F_NRG = []

for year in range(1995, 2010):
    df_energy = import_energy(year, country_list)
    F_NRG.append(df_energy)

F_NRG_hh

In [11]:
def import_energy_hh(year, country_list):
    array_list = []
    df_list = []
    sheet = str(year)
    
    country = country_list[0]
    file_loc = 'Data/EAS-EU/' + str(country) + '_EU_May12.xls'
    # import data
    df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0).iloc[-2:-1]
    df[str(country)] = df['TOTAL']
    df = df[[str(country)]].fillna(0)
    
    for i in range(1, len(country_list)):
        country = country_list[i]
        file_loc = 'Data/EAS-EU/' + str(country) + '_EU_May12.xls'
        # import data
        temp = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0).iloc[-2:-1]
        temp[str(country)] = temp['TOTAL']
        temp = temp[[str(country)]].fillna(0)
        df = df.join(temp)
        
    return(df)

In [12]:
F_NRG_hh = []

for year in range(1995, 2010):
    df_energy_hh = import_energy_hh(year, country_list)
    F_NRG_hh.append(df_energy_hh)

### Air Quality

F_GHG

In [13]:
# function for Energy Data --> make array of vecotrs containing total energy use per year per sector

def import_air(year, country_list):
    array_list = []
    df_list = []
    sheet = str(year)
    
    country = country_list[0]
    file_loc = 'Data/EAS-AIR/' + str(country) + '_AIR_May12.xls'
    # import data
    df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
    df.index = df.index.droplevel(level=1)
    df = df.join(l1, how='right')
    df = df[['CO2', 'CH4', 'N2O', 'sector']]
    df['CH4'] = (df['CH4'] / 1000) * 28
    df['N2O'] = (df['N2O'] / 1000) * 265
    df['CO2e'] = df['CO2'] + df['CH4'] + df['N2O']
    df['country'] = str(country)
    df = df[['CO2e', 'country', 'sector']].fillna(0)
    
    for i in range(1, len(country_list)):
        country = country_list[i]
        file_loc = 'Data/EAS-AIR/' + str(country) + '_AIR_May12.xls'
        # import data (only from 35 sectors, remove totals)
        temp = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
        temp.index = temp.index.droplevel(level=1)
        temp = temp.join(l1, how='right')
        temp = temp[['CO2', 'CH4', 'N2O', 'sector']]
        # convert to kilotonnes and then construct CO2e (using 100 year GWP, AR5; https://www.ghgprotocol.org/sites/default/files/ghgp/Global-Warming-Potential-Values%20%28Feb%2016%202016%29_1.pdf)
        temp['CH4'] = (temp['CH4'] / 1000) * 28
        temp['N2O'] = (temp['N2O'] / 1000) * 265
        temp['CO2e'] = temp['CO2'] + temp['CH4'] + temp['N2O']
        temp['country'] = str(country)
        temp = temp[['CO2e', 'country', 'sector']].fillna(0)
    
        df = df.append(temp)
        
    df = df.set_index(['country', 'sector']).T
    return(df)

In [14]:
F_GHG = []

for year in range(1995, 2010):
    df_air = import_air(year, country_list)
    F_GHG.append(df_air)

F_GHG_hh

In [15]:
def import_air_hh(year, country_list):
    array_list = []
    df_list = []
    sheet = str(year)
    
    country = country_list[0]
    file_loc = 'Data/EAS-AIR/' + str(country) + '_AIR_May12.xls'
    # import data
    df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0).iloc[-2:-1] # select only household category
    df = df[['CO2', 'CH4', 'N2O']]
    df['CH4'] = (df['CH4'] / 1000) * 28
    df['N2O'] = (df['N2O'] / 1000) * 265
    df[str(country)] = df['CO2'] + df['CH4'] + df['N2O']
    df = df[[str(country)]].fillna(0)
    
    for i in range(1, len(country_list)):
        country = country_list[i]
        file_loc = 'Data/EAS-AIR/' + str(country) + '_AIR_May12.xls'
        # import data (only from 35 sectors, remove totals)
        temp = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0).iloc[-2:-1] # select only household category
        temp = temp[['CO2', 'CH4', 'N2O']]
        # convert to kilotonnes and then construct CO2e (using 100 year GWP, AR5; https://www.ghgprotocol.org/sites/default/files/ghgp/Global-Warming-Potential-Values%20%28Feb%2016%202016%29_1.pdf)
        temp['CH4'] = (temp['CH4'] / 1000) * 28
        temp['N2O'] = (temp['N2O'] / 1000) * 265
        temp[str(country)] = temp['CO2'] + temp['CH4'] + temp['N2O']
        temp = temp[[str(country)]].fillna(0)
        df = df.join(temp)
        
    return(df)

In [16]:
F_GHG_hh = []

for year in range(1995, 2010):
    df_air_hh = import_air_hh(year, country_list)
    F_GHG_hh.append(df_air_hh)

# 4. Save files

In [17]:
df_list = ['Z', 'V', 'Y', 'F_NRG', 'F_NRG_hh', 'F_GHG', 'F_GHG_hh']

for df in df_list:
    for i in range(len(eval(df))):
        data = eval(df)[i]
        year = i + 1995
        data.to_csv('Data/clean_data/' + df + '_' + str(year) + '.csv')